## 画像認識モデルからコンテナ作成、Azure Container Registryに登録
- CONDA設定がJETSONにサポートされていないので、このオプションは参考のためです。

In [ ]:
import azureml.core
from azureml.core import Workspace, Dataset, Experiment, Model

In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.location, ws.resource_group, ws.location, sep = '\t')

In [ ]:
model = Model(ws, name="cat-dog-runmodel")

In [ ]:
model.download(target_dir='.', exist_ok=False, exists_ok=None)

### score.py作成

In [ ]:
%%writefile score.py

import json
import numpy as np
from PIL import Image
import tensorflow as tf
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img

def init():
    
    global model
    model = tf.keras.models.load_model("./outputs/keras_simple.h5")

def load_image(filename):
    img = load_img(filename, target_size=(150, 150))
    img = img_to_array(img)
    img = img.reshape(1, 150, 150, 3)
    img = img.astype('float32')
    return img


def run(input_str):
    try:
        input_json = json.loads(input_str)
        print("INPUT_JSON: ", input_json)
        img = load_image(input_json)
        pred = model.predict(img)
        print("Prediction is ", pred[0])
        
        if pred[0] == 1:
            input_json['anomaly']=True
        else:
            input_json['anomaly']=False
        result = [json.dumps(input_json)]
    except Exception as e:
        result = str(e)
    
    return result


## myenv.yml作成

In [ ]:
# This specifies the dependencies to include in the environment
from azureml.core.conda_dependencies import CondaDependencies 
myenv = CondaDependencies.create(
    conda_packages=['tensorflow', 'keras'],
    pip_packages=['pillow', 'numpy']
)

with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())

## score.pyとmyenv.ymlからコンテナイメージ作成

In [ ]:
from azureml.core.image import Image, ContainerImage

image_config = ContainerImage.image_configuration(runtime= "python",
                                 execution_script="score.py",
                                 conda_file="myenv.yml",
                                 tags = {'area': "iot", 'type': "classification"})

image = Image.create(name = "catdogimage",
                     # this is the model object 
                     models = [model],
                     image_config = image_config, 
                     workspace = ws)

In [ ]:
image.wait_for_creation(show_output = True)